In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np

In [2]:
! unzip -q "/content/drive/MyDrive/datasets/animals151.zip"

In [32]:
train_ds = keras.utils.image_dataset_from_directory(
    '/content/dataset/dataset',
    labels="inferred",
    batch_size=32,
    image_size=(224, 224),
    seed=123,
    validation_split=0.2,
    subset='training',
)

validation_ds = keras.utils.image_dataset_from_directory(
    '/content/dataset/dataset',
    labels="inferred",
    batch_size=32,
    image_size=(224, 224),
    seed=123,
    validation_split=0.2,
    subset='validation',
)

Found 6270 files belonging to 151 classes.
Using 5016 files for training.
Found 6270 files belonging to 151 classes.
Using 1254 files for validation.


In [29]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

In [33]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
# create the base pre-trained model
base_model = VGG19(weights='imagenet', include_top=False)
base_model.trainable = False
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [17]:
preprocess_input = keras.applications.vgg19.preprocess_input

In [30]:
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)

x = preprocess_input(x)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  

# maybe overkill
x = layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x) 

outputs = layers.Dense(151, activation='softmax')(x)

vgg19_transfer_model = keras.Model(inputs, outputs)
vgg19_transfer_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem_4   (None, 224, 224, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_4 (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d_6   (None, 512)              0   

In [34]:
vgg19_transfer_model.compile(
    optimizer = keras.optimizers.Adam(1e-3),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ["accuracy"],
)

history = vgg19_transfer_model.fit(train_ds, epochs=20, validation_data=validation_ds)

Epoch 1/20
157/157 [==============================] - 71s 442ms/step - loss: 3.5749 - accuracy: 0.3614 - val_loss: 1.0884 - val_accuracy: 0.7265
Epoch 2/20
157/157 [==============================] - 67s 426ms/step - loss: 1.2977 - accuracy: 0.6730 - val_loss: 0.8657 - val_accuracy: 0.7791
Epoch 3/20
157/157 [==============================] - 67s 426ms/step - loss: 0.9477 - accuracy: 0.7472 - val_loss: 0.8604 - val_accuracy: 0.7911
Epoch 4/20
157/157 [==============================] - 67s 426ms/step - loss: 0.7457 - accuracy: 0.7978 - val_loss: 0.8736 - val_accuracy: 0.7998
Epoch 5/20
157/157 [==============================] - 67s 426ms/step - loss: 0.6088 - accuracy: 0.8297 - val_loss: 0.8168 - val_accuracy: 0.8054
Epoch 6/20
157/157 [==============================] - 67s 426ms/step - loss: 0.5353 - accuracy: 0.8499 - val_loss: 0.8559 - val_accuracy: 0.8142
Epoch 7/20
157/157 [==============================] - 67s 426ms/step - loss: 0.4760 - accuracy: 0.8632 - val_loss: 0.8783 - val_ac

In [35]:
base_model.trainable = True
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [36]:
vgg19_transfer_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 sequential_2 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem_4   (None, 224, 224, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_4 (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d_6   (None, 512)              0   

In [ ]:
vgg19_transfer_model.compile(
    optimizer = keras.optimizers.Adam(1e-5),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ["accuracy"],
)

history_round2 = vgg19_transfer_model.fit(train_ds, epochs=10, validation_data=validation_ds)